**Куски кода, которые по каким-то причинам не нужны/понравились Милене :(**

*exp-1*

In [ ]:
def crop_to_square_opencv(image, face_info, border_pixels=250):
    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']

    square_size = max(w, h) + 2 * border_pixels
    new_x = max(0, x + w // 2 - square_size // 2)
    new_y = max(0, y + h // 2 - square_size // 2)

    cropped_img = image[new_y: new_y + square_size, new_x: new_x + square_size]

    return cropped_img

In [ ]:
def resize_image_opencv(image, new_resolution):
    resized_image = cv2.resize(image, new_resolution)
    return resized_image

In [ ]:
%%time
for model_name in model_names:
    resolution = model_resolutions[model_name]
    df_list = []

    files = glob.glob(os.path.join(folder_path, "**", "*.JPG"), recursive=True)

    # for img_path in tqdm(files, desc=f'Creating images cropped&resized to {resolution} for {model_name}', unit='image'):
    #     rel_path = os.path.relpath(img_path, folder_path)
    #     rel_path = rel_path.replace(os.path.sep, "_")
    #     output_path = os.path.join(tmp_folder, f"{rel_path[:-4]}_{resolution}.jpg")
    #     if not os.path.exists(output_path):
    #         img = cv2.imread(img_path)
    #         faces = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")
    #         for face_info in faces:
    #             cropped_face = crop_to_square(img, face_info)
    #             resized_img = resize_image(cropped_face, (resolution, resolution))
    #             cv2.imwrite(output_path, resized_img)
           
    image_files = [f for f in os.listdir(tmp_folder) if f.endswith(f'_{resolution}.jpg')]
    
    for filename1 in tqdm(image_files, desc=f'Processing with {model_name}', unit='image'):
        img_path1 = os.path.join(tmp_folder, filename1)
        # img_1 = cv2.imread(img_path1)
        img_1 = Image.open(img_path1)
        img_1 = np.array(img_1)
        for filename2 in  tqdm(image_files, desc=f'Comparing all faces to {filename1}', unit='image'):
            img_path2 = os.path.join(tmp_folder, filename2)

            # img_2 = cv2.imread(img_path2)
            img_2 = Image.open(img_path2)
            img_2 = np.array(img_2)
            # https://stackoverflow.com/questions/74567500/tensorflow-unknownerror-graph-execution-error-jit-compilation-failed-op-in
            # with tf.device('/CPU:0'): 
            start_time = time.perf_counter()
            result = DeepFace.verify(
                img1_path=img_1,
                img2_path=img_2,
                enforce_detection=False,
                detector_backend="retinaface",
                model_name=model_name,
                distance_metric="euclidean_l2"
            )
            # print(f"time: {time.perf_counter() - start_time}")
            # print(f"distance for {img_path2}: {result['distance']}")
            img1_name = os.path.splitext(filename1)[0].rsplit('_', 1)[0]
            img2_name = os.path.splitext(filename2)[0].rsplit('_', 1)[0]

            df_list.append(pd.DataFrame({'img_1': [img1_name], 'img_2': [img2_name], 'distance': [result['distance']], 'verified': [result['verified']]}))
            
        df = pd.concat(df_list, ignore_index=True)
        
    csv_file_path = os.path.join(tmp_folder, f'{model_name}_comparison_results.csv')
    df.to_csv(csv_file_path, index=False)

In [ ]:
for model_name in model_names:
    resolution = model_resolutions[model_name]
    df_list = []

    files = glob.glob(os.path.join(folder_path, "**", "*.JPG"), recursive=True)
    # print(files)
    for img_path in tqdm(files, desc=f'Creating images cropped&resized to {resolution} for {model_name}', unit='image'):
        rel_path = os.path.relpath(img_path, folder_path)
        rel_path = rel_path.replace(os.path.sep, "_")
        output_path = os.path.join(tmp_folder, f"{rel_path[:-4]}_{resolution}.jpg")
        # print(f"{output_path}")
        if not os.path.exists(output_path):
            image = Image.open(img_path)
            image = ImageOps.exif_transpose(image)
            # display(image)
            faces = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")
            for face_info in faces:
                cropped_face = crop_to_square_pil(image, face_info)
                resized_img = resize_image_pil(cropped_face, (resolution, resolution))
                # display(resized_img)
                # time.sleep(3)
                resized_img.save(output_path)

*exp-2*

In [ ]:
def crop_to_square(image, face_info, border_pixels=0):
    # print(f"Face info: {face_info}")
    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']

    # print(f"x: {x}, y: {y}, w: {w}, h: {h}")
    square_size = max(w, h) + 2 * border_pixels
    # new_x = max(0, x + w // 2 - square_size // 2)
    # new_y = max(0, y + h // 2 - square_size // 2)

    new_x = max(0, x)
    new_y = max(0, y)

    img = Image.fromarray(image)
    cropped_img = img.crop((new_x - border_pixels, new_y - border_pixels, new_x + square_size, new_y + square_size))

    return cropped_img

In [ ]:
def resize_image(image, new_resolution):

    resized_img = image.resize(new_resolution, Image.LANCZOS)
    return resized_img

In [ ]:
for model_name in model_names:
   df_list = []
   for resolution in resolutions:
      
       files = glob.glob(os.path.join(folder_path, "**", "*.JPG"), recursive=True)
    
       for img_path in tqdm(files, desc=f'Creating images cropped&resized to {resolution} for {model_name}', unit='image'):
        rel_path = os.path.relpath(img_path, folder_path)
        rel_path = rel_path.replace(os.path.sep, "_")
        output_path = os.path.join(tmp_folder, f"{rel_path[:-4]}_{resolution}.jpg")
      
        if not os.path.exists(output_path):
            image = Image.open(img_path)
            image = ImageOps.exif_transpose(image)
            
            faces = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")
            for face_info in faces:
                cropped_face = crop_to_square_pil(image, face_info)
                resized_img = resize_image_pil(cropped_face, (resolution, resolution))
                resized_img.save(output_path)

In [ ]:
import ipyplot


files = glob.glob(os.path.join(input_folder, "**", "*.JPG"), recursive=True)

img_path = files[44]
face_info = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")[0]
x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']

image = Image.open(img_path)
# https://github.com/python-pillow/Pillow/issues/4703#issuecomment-645219973
image = ImageOps.exif_transpose(image)
draw = ImageDraw.Draw(image)

image_array = np.array(image)
image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
display(image)

cropped_img = crop_to_square(image_array, face_info)
img1 = np.array(cropped_img)

metrics = []
plt.figure()

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(1,2) 

for resolution in resolutions:
    resized_img = resize_image(cropped_img, (resolution, resolution))
    img2 = np.array(resized_img)
    axarr[0].imshow(img1)
    axarr[1].imshow(img2)
    plt.show()
    result = DeepFace.verify(img1_path=img1,
                             img2_path=img2,
                             enforce_detection=False,
                             detector_backend="retinaface",
                             model_name=model_name,
                             distance_metric="euclidean_l2"
                             )
    print(f'distance:{result["distance"]:.5f}')

In [2]:
!unzip "/tf/data/miem_face_recognition/drive-download-20240424T131909Z-001.zip" -d "/tf/data/miem_face_recognition/new_people"

Archive:  /tf/data/miem_face_recognition/drive-download-20240424T131909Z-001.zip
  inflating: /tf/data/miem_face_recognition/new_people/ankarabutova/1K5A9108.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/ankarabutova/1K5A9106.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/ankarabutova/1K5A9109.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/pspopova/1K5A9096.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/pspopova/1K5A9091.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/ankarabutova/1K5A9040.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/pspopova/1K5A9092.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/pspopova/1K5A9073.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/pspopova/1K5A9093.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/pspopova/1K5A9095.JPG  
  inflating: /tf/data/miem_face_recognition/new_people/ankarabutova/1K5A9051.JPG  
  inflating: /tf/data/miem_fac

In [3]:
!unzip "/tf/data/miem_face_recognition/pspopova-20240424T132504Z-001.zip" -d "/tf/data/miem_face_recognition/pspopova"

Archive:  /tf/data/miem_face_recognition/pspopova-20240424T132504Z-001.zip
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/no_glasses(2).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/glasses(6).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/no_glasses(6).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/no_glasses(5).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/no_glasses(3).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/glasses(8).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/glasses(4).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/no_glasses(8).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/glasses(1).JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/glasses.JPG  
  inflating: /tf/data/miem_face_recognition/pspopova/pspopova/no_glasses(4).JPG  
  inflating: /tf/data/miem_face_recogniti

exp-3

In [ ]:
import random

users_list = os.listdir(os.path.join(input_folder, f"center/center/"))
horizontal_positions = ["45_left","center","45_right"]
vertical_positions = ["up","center","down"]

model_name = "VGG-Face"

for person_name in tqdm(users_list):
    person_distances = {"up": {"45_left": 0, "center": 0, "45_right": 0},
                        "center": {"45_left": 0, "center": 0, "45_right": 0},
                        "down": {"45_left": 0, "center": 0, "45_right": 0}}
    image_anfas_no_glasses_path = os.path.join(input_folder, f"center/center/{person_name}/no_glasses.JPG")
    image_anfas_no_glasses = Image.open(image_anfas_no_glasses_path)
    image_anfas_no_glasses = ImageOps.exif_transpose(image_anfas_no_glasses)
    display(image_anfas_no_glasses.resize((128, 128)))
    image_anfas_no_glasses = np.array(image_anfas_no_glasses)
    
    image_anfas_glasses_path = os.path.join(input_folder, f"center/center/{person_name}/glasses.JPG")
    yesGlasses =  os.path.exists(image_anfas_glasses_path)
    for vertical_position in vertical_positions:
        for horizontal_position in horizontal_positions:
            if vertical_position == horizontal_position:
                continue
            image_current_no_glasses_path = os.path.join(input_folder, f"{vertical_position}/{horizontal_position}/{person_name}/no_glasses.JPG")
            image_current_no_glasses = Image.open(image_current_no_glasses_path)
            image_current_no_glasses = ImageOps.exif_transpose(image_current_no_glasses)
            # ipyplot.plot_images([image_anfas_no_glasses, image_current_no_glasses], max_images=20, img_width=150)
            # display(image_current_no_glasses.resize((128, 128)))
            image_current_no_glasses = np.array(image_current_no_glasses)

            result = DeepFace.verify(
                img1_path=image_anfas_no_glasses,
                img2_path=image_current_no_glasses,
                enforce_detection=True,
                detector_backend="mediapipe",
                # detector_backend="ssd",
                model_name=model_name,
                distance_metric="euclidean_l2"
            )
            # print(f"dry run: comparing {vertical_position}/{horizontal_position}/{person_name}/no_glasses.JPG with center/center/{person_name}/glasses.JPG")
            print(f"real run: comparing {vertical_position}/{horizontal_position}/{person_name}/no_glasses.JPG with center/center/{person_name}/glasses.JPG, distance is {result['distance']}")
            person_distances[vertical_position][horizontal_position] = result['distance']
            if yesGlasses:
                image_current_glasses_path = os.path.join(input_folder, f"{vertical_position}/{horizontal_position}/{person_name}/glasses.JPG")               
                image_current_glasses = Image.open(image_current_glasses_path)
                image_current_glasses = ImageOps.exif_transpose(image_current_glasses)
                # ipyplot.plot_images([image_anfas_no_glasses, image_current_no_glasses], max_images=20, img_width=150)
                # display(image_current_glasses.resize((128, 128)))
                image_current_glasses = np.array(image_current_glasses)
            print(f'{horizontal_position}_{vertical_position}')
        # print(np.average(person_distances[vertical_position][horizontal_position]))
        # print(list(person_distances[vertical_position].values()))
        # print(np.average(list(person_distances[vertical_position].values())))
        # print(np.median(list(person_distances[vertical_position].values())))
        df_results.append(pd.DataFrame({'img_1': [img1_name], 'img_2': [img2_name], 'distance': [result['distance']], 'verified': [result['verified']]}))
    time.sleep(5)

In [15]:
!pip install --upgrade jupyterlab-git